In [1]:
import alc
from src.alc import calcularPerformanceDeCalificacion

### Cargar dataset

In [2]:
Xt, Yt, Xv, Yv = alc.cargarDataset("./cats_and_dogs")

### Descomposiciones

#### Cholesky

In [4]:
WCholesky = alc.algoritmo1(Xt, Yt)
MCholesky = WCholesky@Xv
confusionCholesky = alc.matrizConfusion(MCholesky, Yv)
confusionCholesky

Calculando LU para matriz de:  (1536, 2000)
Calculando LU para matriz de:  (1536, 1536)
Calculando LU para matriz de:  (1536, 1536)


array([[334., 166.],
       [150., 350.]])

#### Valores Singulares

In [4]:
WSVD = alc.algoritmo2(Xt, Yt, svdRango=20)
MSVD = WSVD@Xv
confusionSVD = alc.matrizConfusion(MSVD, Yv)
confusionSVD

svd_reducida es llamada desde algoritmo2
traspuesta   es llamada desde svd_reducida
multiplicar  es llamada desde svd_reducida
diagRH       es llamada desde svd_reducida
metpot2k     es llamada desde diagRH, faltando 20 recursiones de diagRH
metpot2k     aplica matriz cantidad de veces: 
    10
metpot2k     obtuvo autovalor: 53055.08092305524
metpot2k     es llamada desde diagRH, faltando 19 recursiones de diagRH
metpot2k     aplica matriz cantidad de veces: 
    2000
metpot2k     obtuvo autovalor: 971.892205931802
pinvSVD      es llamada desde algoritmo2


array([[317., 183.],
       [274., 226.]])

#### QR

##### Householder

In [3]:
WQR = alc.algoritmo3(Xt, Yt)
MQR = WQR@Xv
confusionQR = alc.matrizConfusion(MQR, Yv)
confusionQR

Calculando QR con HH
	Iteracion: 1535
pinvHouseHolder
	calculando vector: 1999
calculando W


array([[334., 166.],
       [150., 350.]])

##### Gram-Schmidt

In [3]:
WQR2 = alc.algoritmo3(Xt, Yt, metodo="GS")
MQR2 = WQR2@Xv
confusionQR2 = alc.matrizConfusion(MQR2, Yv)
confusionQR2

Calculando QR con GS no cuadrada
Calculando columna: 1535
pinvGramSchmidt
	calculando vector: 1999
calculando W


array([[334., 166.],
       [150., 350.]])

#### Tablas de aciertos por algoritmo

In [4]:
# calificacionCh = calcularPerformanceDeCalificacion(confusionCholesky)
# calificacionSVD = calcularPerformanceDeCalificacion(confusionSVD)
calificacionHH = calcularPerformanceDeCalificacion(confusionQR)
# calificacionGS = calcularPerformanceDeCalificacion(confusionQR2)

In [5]:
tabla = [
    [calificacionHH[0]],
    [calificacionHH[1]]
]

columnas = ["Cholesky", "SVD", "HH", "GS"]
filas = ["% correctas gatos", "% correctas perros"]

(np.float64(0.668), np.float64(0.7))

## Experimentacion